In [8]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 300)

from pickle import load
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

In [9]:
final_scaler = load(open('scaler.pickle','rb'))
final_model = load(open('model.pickle','rb'))

In [17]:
df = pd.read_csv('data/holdout_data.csv')
columns = pd.Series(['ID','LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'])

df.columns = columns
df.drop(columns=['ID'],inplace=True)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7501 entries, 0 to 7500
Data columns (total 23 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   LIMIT_BAL  7501 non-null   int64
 1   SEX        7501 non-null   int64
 2   EDUCATION  7501 non-null   int64
 3   MARRIAGE   7501 non-null   int64
 4   AGE        7501 non-null   int64
 5   PAY_1      7501 non-null   int64
 6   PAY_2      7501 non-null   int64
 7   PAY_3      7501 non-null   int64
 8   PAY_4      7501 non-null   int64
 9   PAY_5      7501 non-null   int64
 10  PAY_6      7501 non-null   int64
 11  BILL_AMT1  7501 non-null   int64
 12  BILL_AMT2  7501 non-null   int64
 13  BILL_AMT3  7501 non-null   int64
 14  BILL_AMT4  7501 non-null   int64
 15  BILL_AMT5  7501 non-null   int64
 16  BILL_AMT6  7501 non-null   int64
 17  PAY_AMT1   7501 non-null   int64
 18  PAY_AMT2   7501 non-null   int64
 19  PAY_AMT3   7501 non-null   int64
 20  PAY_AMT4   7501 non-null   int64
 21  PAY_AMT5   750

In [20]:
conditions = [df.EDUCATION == 0,
             df.EDUCATION == 6,
             df.EDUCATION == 5]

choices = [4,4,4]

df.EDUCATION = np.select(conditions,choices,default=df.EDUCATION)


In [21]:
df.MARRIAGE = np.where(df.MARRIAGE ==0,3,df.MARRIAGE)
df.MARRIAGE.value_counts()

2    3938
1    3464
3      99
Name: MARRIAGE, dtype: int64

In [22]:
for i in range(6):
    df[f'BILL_AMT{i+1}'] = np.where(df[f'BILL_AMT{i+1}'] > df[f'BILL_AMT{i+1}'].quantile(.99),
                                    df[f'BILL_AMT{i+1}'].quantile(.99),
                                    df[f'BILL_AMT{i+1}']
                                   )
    df[f'PAY_AMT{i+1}'] = np.where(df[f'PAY_AMT{i+1}'] > df[f'PAY_AMT{i+1}'].quantile(.99),
                                    df[f'PAY_AMT{i+1}'].quantile(.99),
                                    df[f'PAY_AMT{i+1}']
                                   )

df.LIMIT_BAL = np.where(df.LIMIT_BAL > df.LIMIT_BAL.quantile(.99),
                        df.LIMIT_BAL.quantile(.99),
                       df.LIMIT_BAL)

In [23]:
for i in range(6):
    df[f'percent_use{i+1}'] = df[f'BILL_AMT{i+1}']/df['LIMIT_BAL']

In [24]:
df['high_lim'] = np.where(df.LIMIT_BAL > df.LIMIT_BAL.quantile(.85),
                         df.LIMIT_BAL.quantile(.85),
                         df.LIMIT_BAL)

In [25]:
for i in range(6):
    df[f'late{i+1}'] = np.where(df[f'PAY_{i+1}'] > 1, 1, 0)

In [26]:
for i in range(6):
    df[f'full{i+1}'] = np.where(df[f'BILL_AMT{i+1}'] - df[f'PAY_AMT{i+1}'] <= 0, 1, 0)

In [27]:
for i in range(6):
    df[f'percent_pay{i+1}'] = df[f'PAY_AMT{i+1}'] / df[f'BILL_AMT{i+1}']
    df[f'percent_pay{i+1}'] = np.where(df[f'BILL_AMT{i+1}'] <= 0,1,df[f'percent_pay{i+1}'])

In [28]:
df['young'] = np.where(df.AGE <= 30, 1, 0)

In [29]:
df = pd.get_dummies(df,columns=['SEX','EDUCATION','MARRIAGE'],drop_first=True)

In [30]:
df.describe()

,LIMIT_BAL,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,percent_use1,percent_use2,percent_use3,percent_use4,percent_use5,percent_use6,high_lim,late1,late2,late3,late4,late5,late6,full1,full2,full3,full4,full5,full6,percent_pay1,percent_pay2,percent_pay3,percent_pay4,percent_pay5,percent_pay6,young,SEX_2,EDUCATION_2,EDUCATION_3,EDUCATION_4,MARRIAGE_2,MARRIAGE_3
count,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000
mean,168088.254899,35.598187,-0.024530,-0.148647,-0.184242,-0.238901,-0.285695,-0.301560,49790.493801,47780.603386,46125.241834,42230.583922,39479.332756,37825.890815,5086.511932,5016.726037,4557.735502,4367.027863,4010.188908,4367.197440,0.419528,0.408252,0.390488,0.356873,0.331282,0.317527,154162.111718,0.101053,0.143714,0.136648,0.113185,0.093321,0.101320,0.214105,0.235969,0.233436,0.244634,0.251966,0.274897,0.894823,1.002991,1.369645,0.971744,0.852410,1.222261,0.359952,0.605253,0.468471,0.160512,0.015598,0.524997,0.013198
std,127214.876266,9.158158,1.125669,1.194837,1.190707,1.151804,1.112400,1.137123,67347.081786,64800.354077,63526.092045,59198.268381,55823.127968,54525.740904,9577.841536,10198.524347,9758.112455,9264.735214,8269.533281,10142.965667,0.405040,0.397264,0.387604,0.367704,0.347780,0.346062,100592.815431,0.301419,0.350823,0.343499,0.316840,0.290901,0.301772,0.410227,0.424631,0.423045,0.429899,0.434171,0.446492,5.348488,7.159638,22.384655,6.757505,5.507296,13.119356,0.480018,0.488829,0.499038,0.367105,0.123922,0.499408,0.114131
min,10000.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-154973.000000,-33350.000000,-24702.000000,-170000.000000,-20753.000000,-73895.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.619892,-0.298900,-0.075525,-1.374500,-0.108650,-0.427280,10000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,60000.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3658.000000,3016.000000,2863.000000,2342.000000,1876.000000,1315.000000,993.000000,856.000000,390.000000,316.000000,272.000000,192.000000,0.020571,0.017921,0.016022,0.013744,0.011194,0.008000,60000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.039054,0.038405,0.033253,0.033305,0.036050,0.035930,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,140000.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22358.000000,21465.000000,20344.000000,19097.000000,18378.000000,17352.000000,2122.000000,2042.000000,1800.000000,1551.000000,1579.000000,1502.000000,0.307700,0.293500,0.271044,0.236320,0.211717,0.186400,140000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.077049,0.076206,0.060387,0.055562,0.060516,0.067281,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,240000.000000,42.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67722.000000,64697.000000,60185.000000,54589.000000,50149.000000,49104.000000,5057.000000,5000.000000,4664.000000,4200.000000,4000.000000,4000.000000,0.822800,0.804000,0.754485,0.661308,0.5

In [33]:
scaled_df = final_scaler.transform(df)

In [35]:
preds = final_model.predict(scaled_df)

In [36]:
final_pred = pd.DataFrame(data=preds,columns=['default'])



In [37]:
final_pred

,default
0,0
1,0
2,1
3,0
4,1
...,...
7496,0
7497,0
7498,1
7499,0


In [39]:
final_pred.to_csv('preds_Mitch_Krieger.csv')

default
0          5545
1          1956
dtype: int64